In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Data Ingestion Techniques
loader - UnstructuredLoader

In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader
url = ["https://www.paloaltonetworks.ca/cyberpedia/ai-governance"]
reader = UnstructuredURLLoader(urls=url)
doc = reader.load()
doc

# Chunking or Splitting...

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_doc = text_splitter.split_documents(documents=doc)
splitted_doc

# Embeddings
Vector Store - Chroma DB 

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents=splitted_doc, embedding=OpenAIEmbeddings())

## Integrate LLM

In [7]:
from langchain_openai import OpenAI
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_retries=2
)

## Design Prompt Template

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>                                                                                                                                                                
Question: {input}
""")

## Chain
create stuff document chain

In [9]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm=llm,prompt=prompt)

## Retrievers

In [10]:
retriever = db.as_retriever(search_type="mmr")

### Retrieval chain(Retriever + Document chain):
    Retrieval chain takes in a user query, which is then passed to the retriever to fetch relevant documents. Those documents and original inputs are then passed to an LLM to generate a response.


In [11]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
response = retrieval_chain.invoke({"input": "What is the differnce b/w explainability and explainable AI?"})

In [21]:
response['answer']

'Answer: Explainability is the desired characteristic of an AI system, while explainable AI is the field of study and practice that aims to achieve this characteristic in AI models. In other words, explainability is the end goal, while explainable AI is the means to achieve it.'